<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import sqlalchemy
import psycopg2
import numpy as np

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [100]:
# текст запроса
query_3_1 = f'''select
            count(*)
            from vacancies
'''
df = pd.read_sql_query(query_3_1, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,49197


In [9]:
# результат запроса
49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [89]:
# текст запроса
query_3_2 = f'''select
            count(*)
            from employers
'''
df = pd.read_sql_query(query_3_2, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,23501


In [11]:
# результат запроса
23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [90]:
# текст запроса
query_3_3 = f'''select
            count(*)
            from areas
'''
df = pd.read_sql_query(query_3_3, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,1362


In [13]:
# результат запроса
1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [91]:
# текст запроса
query_3_4 = f'''select
            count(*)
            from industries
'''
df = pd.read_sql_query(query_3_4, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,294


In [15]:
# результат запроса
294

***

# выводы по предварительному анализу данных
Всего в базе 49197 вакансий в 1632 регионах (населенных пунктах) от 23501 работодателей, по 294 отраслям. Для более подробной информации базы вакансий необходимо провести белее детальный анализ. 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [4]:
# текст запроса (дополнительно определена доля вакансий региона в общем числе вакансий)
query_4_1 = f'''select
            count(v.id) as cnt, /*подсчета значений столбца id таблицы vacancies*/
            round(count(v.id)*100.0 / (select count(*) from vacancies), 2) as percent, /*определение доли каждого города в общем количестве вакансий столбец id таблицы vacancies*/
            a.name as area
            from vacancies v
            join areas a on v.area_id = a.id /*присоединение таблицы areas a; условие: area_id таблицы vacabcies равен id таблицы area*/
            group by a.name /*группировка по регионам*/
            order by count(v.id) desc
            limit 5
'''
df = pd.read_sql_query(query_4_1, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,cnt,percent,area
0,5333,10.84,Москва
1,2851,5.80,Санкт-Петербург
2,2112,4.29,Минск
3,2006,4.08,Новосибирск
4,1892,3.85,Алматы


In [18]:
# результат запроса
# Москва, Санкт-Петербург, Минск, Новосибирск, Алматы

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [36]:
# текст запроса (дополнительно определена доля вакансий с указанием заработной платы в общем числе вакансий)
query_4_2 = f'''select
                count(id) as count, /*подсчет значений столбца id таблицы vacancies*/
                round(count(id)*100.0 / (select count(*) from vacancies), 2) as percent /*определение доли вакансий id с информацией по заработной плате таблицы vacancies*/
            from vacancies
            where (salary_from is not null) or (salary_to is not null) /*условие фильтрации поле salary_from не равное null или поле salary_to не равное null*/
'''
df = pd.read_sql_query(query_4_2, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count,percent
0,24073,48.93


In [20]:
# результат запроса
# 24073
# 51.07 % вакансий без указания заработной платы

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [115]:
# текст запроса
query_4_3 = f'''select
                round(avg(salary_from)) as avg_salary_from, /*расчет среднего значения столбца salary_from*/
                round(avg(salary_to)) as avg_salary_to /*расчет среднего значения столбца salary_to*/
            from vacancies
            '''
df = pd.read_sql_query(query_4_3, connection)
df

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


In [ ]:
# результат запроса
# нижняя граница зарплатной вилки - 71 065 руб.
# верхняя граница зарплатной вилки - 110 537 руб.

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [37]:
# текст запроса (дполнительно опрелена доля каждого сочетания в общем количесвте вакансий)
query_4_4 = f'''select
                count(id) as count, /*подсчет значений столбца id таблицы vacancies*/
                round(count(id)*100.0 / (select count(*) from vacancies), 2) as percent, /*определение доли вакансий id таблицы vacancies*/
                schedule as schedule,
                employment as employment
            from vacancies
            group by schedule, employment /*группировка по столбцам schedule, employment*/
            order by count desc
            '''
df = pd.read_sql_query(query_4_4, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count,percent,schedule,employment
0,35367,71.89,Полный день,Полная занятость
1,7802,15.86,Удаленная работа,Полная занятость
2,1593,3.24,Гибкий график,Полная занятость
3,1312,2.67,Удаленная работа,Частичная занятость
4,940,1.91,Сменный график,Полная занятость
5,569,1.16,Полный день,Стажировка
6,367,0.75,Вахтовый метод,Полная занятость
7,347,0.71,Полный день,Частичная занятость
8,312,0.63,Гибкий график,Частичная занятость
9,141,0.29,Полный день,Проектная работа


In [24]:
# результат запроса
# удаленная работа - полная занятость - 71.89%

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [38]:
# текст запроса (дополнительно определна доля каждого типа требуемого опыта в общем количестве вакансий)
query_4_5 = f'''select
                count(id) as count, /*подсчет значений столбца id таблицы vacancies*/
                round(count(id)*100.0 / (select count(*) from vacancies), 2) as percent,/*определение доли вакансий id таблицы vacancies*/
                experience as experience
            from vacancies
            group by experience /*группировка по столбцу experience*/
            order by count
            '''
df = pd.read_sql_query(query_4_5, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count,percent,experience
0,1337,2.72,Более 6 лет
1,7197,14.63,Нет опыта
2,14511,29.50,От 3 до 6 лет
3,26152,53.16,От 1 года до 3 лет


In [26]:
# результат запроса
# Более 6 лет - Нет опыта - От 3 до 6 лет - От 1 года до 3 лет
# Требуемый опыт работы от 1 года до 6 лет для 82,63% вакансий

In [13]:
# анализ распределения вакансий по отралям
query_4_6 = f'''select
            i.name as industry,
            count(v.id) as count, /*подсчет значений столбца id таблицы vacancies*/
            round(count(v.id) * 100.0 / (select count(*) from vacancies),2) as percent /*определение доли вакансий id таблицы vacancies*/
            from vacancies v
            left join employers_industries ei on v.employer_id = ei.employer_id /* присоединение таблицы employers_industries ei; условие: employer_id  равно employer_id*/
            left join industries i on ei.industry_id = i.id /* присоединение таблицы industries e; условие: industry_id  равно id*/
            group by i.name /*группировка по столбцу name*/
            order by count(v.id) desc
            limit 10
            '''
df = pd.read_sql_query(query_4_6, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,industry,count,percent
0,Разработка программного обеспечения,12499,25.41
1,"Системная интеграция, автоматизации технологи...",11034,22.43
2,None,10084,20.50
3,"Интернет-компания (поисковики, платежные систе...",6413,13.04
4,Банк,2742,5.57
5,Мобильная связь,1585,3.22
6,Кадровые агентства,1143,2.32
7,Консалтинговые услуги,1137,2.31
8,Интернет-провайдер,1095,2.23
9,Розничная сеть (продуктовая),1034,2.10


In [5]:
# дополнительно, проанализировали среднюю заработную плату 
# по отраслям (вакансии, где не указана информация по заработной плате, исключили)
query_4_7 = f'''select
            i.name as industry,
            count(v.id) as count, /*подсчет значений столбца id таблицы vacancies*/
            round(avg(coalesce((v.salary_from + v.salary_to)/2, v.salary_to, v.salary_from))) as avg_salary /*расчет среднего значения заработной платы*/
            from vacancies v
            left join employers_industries ei on v.employer_id = ei.employer_id /* присоединение таблицы employers_industries ei; условие: employer_id  равно employer_id*/
            left join industries i on ei.industry_id = i.id /* присоединение таблицы industries i; условие: industry_id  равно id*/
            where v.salary_from is not null or v.salary_to is not null /*фильтрация по полю salary_from не равное null или поле salary_to не равное null*/
            group by i.name /*группировка по столбцу name*/
            order by avg_salary desc 
            limit 20
            '''
df = pd.read_sql_query(query_4_7, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,industry,count,avg_salary
0,"Агрохимия (продвижение, оптовая торговля)",14,381429.0
1,Агрохимия (производство),16,339875.0
2,Игорный бизнес,20,191905.0
3,Корма для животных (производство),29,168300.0
4,Зоомагазин,31,167613.0
5,Кадровые агентства,711,144530.0
6,"Лакокрасочная продукция, сырье (производство)",26,142803.0
7,"Бытовая химия, парфюмерия, косметика (произво...",78,130120.0
8,"Лакокрасочная продукция, сырье (продвижение, о...",32,128445.0
9,"Дорожно-строительная техника, сельскохозяйстве...",90,122878.0


In [17]:
# Анализ структуры вакансий в отраслях "1. Агрохимия (продвижение, оптовая торговля)" и "2. Агрохимия (производство)"
query_4_7_1 = f'''select
            i.name as industry,
            v.name as name,
            coalesce((v.salary_from + v.salary_to)/2, v.salary_to, v.salary_from, 0) as avg_salary
            from vacancies v
            left join employers_industries ei on v.employer_id = ei.employer_id
            left join industries i on ei.industry_id = i.id
            where i.name like 'Агрохимия%' 
            and (v.salary_from is not null or v.salary_to is not null) /*фильтрация по полю наименование (таблица industries) начинается с "Агрохимия" и в таблице vacancies поля salary_from или поле salary_to не равно null*/
            order by industry asc, avg_salary desc
            '''
df = pd.read_sql_query(query_4_7_1, connection)
df

,industry,name,avg_salary
0,"Агрохимия (продвижение, оптовая торговля)",Директор научно-исследовательского центра (химия),500000
1,"Агрохимия (продвижение, оптовая торговля)",Директор научно-исследовательского центра (химия),500000
2,"Агрохимия (продвижение, оптовая торговля)",Директор научно-исследовательского центра (химия),500000
3,"Агрохимия (продвижение, оптовая торговля)",Директор научно-исследовательского центра (химия),500000
4,"Агрохимия (продвижение, оптовая торговля)",Директор научно-исследовательского центра (химия),500000
5,"Агрохимия (продвижение, оптовая торговля)",Директор научно-исследовательского центра (химия),500000
6,"Агрохимия (продвижение, оптовая торговля)",Директор научно-исследовательского центра (химия),500000
7,"Агрохимия (продвижение, оптовая торговля)",Директор научно-исследовательского центра (химия),500000
8,"Агрохимия (продвижение, оптовая торговля)",Директор научно-исследовательского центра (химия),500000
9,"Агрохимия (продвижение, оптовая торговля)",Директор научно-исследовательского центра (химия),500000


In [18]:
# Анализ структуры вакансий в отраслях "3. Игорный бизнес"
query_4_7_2 = f'''select
            i.name as industry,
            v.name as name,
            coalesce((v.salary_from + v.salary_to)/2, v.salary_to, v.salary_from, 0) as avg_salary
            from vacancies v
            left join employers_industries ei on v.employer_id = ei.employer_id
            left join industries i on ei.industry_id = i.id
            where i.name like '%Игорный бизнес%' 
            and (v.salary_from is not null or v.salary_to is not null) /*фильтрация по полю наименование (таблица industries) содержит "Игорный бизнес" и в таблице vacancies поля salary_from или поле salary_to не равно null*/
            order by avg_salary desc
            '''
df = pd.read_sql_query(query_4_7_2, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,industry,name,avg_salary
0,Игорный бизнес,Android разработчик (Senior/ Middle+),400000
1,Игорный бизнес,IOS-developer (Swift)_Senior/Middle+,400000
2,Игорный бизнес,Backend разработчик / PHP developer,350000
3,Игорный бизнес,Ведущий разработчик баз данных Oracle Pl/SQL,300000
4,Игорный бизнес,Руководитель проектов ИТ,250000
5,Игорный бизнес,Frontend-разработчик,240000
6,Игорный бизнес,Frontend-разработчик,240000
7,Игорный бизнес,Frontend-разработчик,240000
8,Игорный бизнес,Frontend-разработчик,240000
9,Игорный бизнес,QA Engineer / Специалист по тестированию (Midd...,220000


In [19]:
# Анализ структуры вакансий в отраслях "4. Корма для животных (производство)"
query_4_7_3 = f'''select
            i.name as industry,
            v.name as name,
            coalesce((v.salary_from + v.salary_to)/2, v.salary_to, v.salary_from, 0) as avg_salary
            from vacancies v
            left join employers_industries ei on v.employer_id = ei.employer_id
            left join industries i on ei.industry_id = i.id
            where i.name like '%Корма для животных (производство)%' 
            and (v.salary_from is not null or v.salary_to is not null) /*фильтрация по полю наименование (таблица industries) содержит "Корма для животных (производство)" и в таблице vacancies поля salary_from или поле salary_to не равно null*/
            order by avg_salary desc
            '''
df = pd.read_sql_query(query_4_7_3, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,industry,name,avg_salary
0,Корма для животных (производство),Аналитик 1С,210000
1,Корма для животных (производство),Middle PHP разработчик (middle backend php dev...,206500
2,Корма для животных (производство),Middle PHP разработчик (middle backend php dev...,206500
3,Корма для животных (производство),Middle PHP разработчик (middle backend php dev...,206500
4,Корма для животных (производство),Middle PHP разработчик (middle backend php dev...,206500
5,Корма для животных (производство),Middle PHP разработчик (middle backend php dev...,206500
6,Корма для животных (производство),Middle PHP разработчик (middle backend php dev...,206500
7,Корма для животных (производство),Middle PHP разработчик (middle backend php dev...,206500
8,Корма для животных (производство),Middle PHP разработчик (middle backend php dev...,206500
9,Корма для животных (производство),Middle PHP разработчик (middle backend php dev...,206500


In [20]:
# Анализ структуры вакансий в отраслях "5. Зоомагазин"
query_4_7_4 = f'''select
            i.name as industry,
            v.name as name,
            coalesce((v.salary_from + v.salary_to)/2, v.salary_to, v.salary_from, 0) as avg_salary
            from vacancies v
            left join employers_industries ei on v.employer_id = ei.employer_id
            left join industries i on ei.industry_id = i.id
            where i.name like '%Зоомагазин%' 
            and (v.salary_from is not null or v.salary_to is not null) /*фильтрация по полю наименование (таблица industries) содержит "Зоомагазин" и в таблице vacancies поля salary_from или поле salary_to не равно null*/
            order by avg_salary desc
            '''
df = pd.read_sql_query(query_4_7_4, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,industry,name,avg_salary
0,Зоомагазин,Аналитик 1С,210000
1,Зоомагазин,Middle PHP разработчик (middle backend php dev...,206500
2,Зоомагазин,Middle PHP разработчик (middle backend php dev...,206500
3,Зоомагазин,Middle PHP разработчик (middle backend php dev...,206500
4,Зоомагазин,Middle PHP разработчик (middle backend php dev...,206500
5,Зоомагазин,Middle PHP разработчик (middle backend php dev...,206500
6,Зоомагазин,Middle PHP разработчик (middle backend php dev...,206500
7,Зоомагазин,Middle PHP разработчик (middle backend php dev...,206500
8,Зоомагазин,Middle PHP разработчик (middle backend php dev...,206500
9,Зоомагазин,Middle PHP разработчик (middle backend php dev...,206500


***

# выводы по детальному анализу вакансий
Всего вакансий 49 197, больше всего вакансий (более 2000) в 4-х крупных городах:
* Москва, 
* Санкт-Петербург, 
* Минск, 
* Новосибирск. 

У более чем половины вакансий какая-либо информация по уровню предлагаемой заработной платы не указана. В тех вакансиях, где заработная плата указана «вилка» составляет 71 065 (руб.) – нижняя граница, 110 573 руб. – верхняя граница. 

Более 70 % вакансий предполагают полную занятость и полный рабочий день.

Для 80% требуется опыт работы от 1 года до 6 лет. 

Около половины вакансий – это отрасли, связанные напрямую с ИТ: 25,4% от общего числа - разработка ПО, 22,4 % - системная интеграция, автоматизация.

Для вакансий, с указанием верхней, либо нижней границы заработной платы, дополнительно, проанализировали среднюю заработную плату по отраслям. Топ 5 высокооплачиваемых отраслей: 
* "Агрохимия (продвижение, оптовая торговля)" – 381 429 руб., 
* "Агрохимия (производство)" – 339 875 руб., 
* "Игорный бизнес" – 191 905 руб., 
* "Корма для животных (производство)" – 168 300 руб., 
* "Зоомоагазин"  – 167 613 руб.

Для данных отраслей дополнительно проанализировали структуру вакансий. Высокий уровень заработной платы в "Агрохимии" связан с наличием нескольких вакансий Директора научно-исследовательского центра, но большая часть вакансий в отраслях с наибольшей заработной платой связана с разработкой ПО, соответственно, можно предположить, что уровень заработной платы не зависит от сферы деятельности компании. 


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [4]:
# текст запроса
query_5_1 = f'''select
            count(v.id) as count, /*подсчет значений столбца id таблицы vacancies*/
            e.name as name
            from vacancies v
            join employers e on v.employer_id = e.id /* присоединение таблицы employers e; условие: employer_id  равно id*/
            group by e.name /*группировка по столбцу name таблицы employers*/
            order by count(v.id) desc
            limit 10
            '''
df = pd.read_sql_query(query_5_1, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть
5,327,ИК СИБИНТЕК
6,292,МТС
7,247,DataArt
8,204,Совкомбанк Технологии
9,176,Первый Бит


In [29]:
# результат запроса
# Яндекс, Газпром нефть

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [32]:
# текст запроса
query_5_2 = f'''select
            a.name as area,    
            count(v.id) as vacancies_count, /*подсчет значений столбца id таблицы vacancies*/
            count(distinct e.id) as employers_count /*подсчет уникальных значений столбца id таблицы employers*/
            from areas a
            left join employers e on a.id = e.area /*присоединение таблицы employers e; условие: area равно id*/
            left join vacancies v on v.area_id = a.id /*присоединение таблицы vacancies v; условие: area_id равно id*/
            where v.id is null /*фильтрации по полю id (таблицы vacancies) равно null*/
            group by a.name /*группировка по столбцу name таблицы areas*/
            order by employers_count desc
            '''
df = pd.read_sql_query(query_5_2, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,area,vacancies_count,employers_count
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
...,...,...,...
588,Кувандык,0,0
589,Куйбышев,0,0
590,Куйбышево (Ростовская область),0,0
591,Култаево,0,0


In [31]:
# результат запроса
# Россия

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [34]:
# текст запроса
query_5_3 = f'''select
            e.name as employer,
            count(distinct v.area_id) as area_count   /*подсчет уникальных значений столбца area_id таблицы vacancies*/ 
            from employers e
            left join vacancies v on e.id = v.employer_id /*присоединение таблицы vacancies v; условие: employer_id равно id*/
            group by e.name /*группировка по столбцу name таблицы employers*/
            order by area_count desc
            limit 10
            '''
df = pd.read_sql_query(query_5_3, connection)
df

,employer,area_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


In [33]:
# результат запроса
# 181

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [36]:
# текст запроса
query_5_4 = f'''select
            count(distinct e.id) as employers_count /*подсчет уникальных значений столбца id таблицы employers*/
            from employers e
            left join employers_industries ei on e.id = ei.employer_id /*присоединение таблицы employers_industries ei; условие: employer_id  равно id*/
            where ei.industry_id is null /*фильтрации по полю industry_id (таблицы employers_industries) равно null*/
            '''
df = pd.read_sql_query(query_5_4, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,employers_count
0,8419


In [35]:
# результат запроса
# 8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [42]:
# текст запроса
query_5_5 = f'''select
            e.name as employer_name,
            count(ei.industry_id) as industry_count /*подсчет уникальных значений столбца industry_id таблицы employers*/
            from employers e
            left join employers_industries ei on e.id = ei.employer_id /*присоединение таблицы employers_industries ei; условие: employer_id  равно id*/
            group by e.name /*группировка по столбцу name таблицы employers*/
            having count(ei.industry_id) = 4   /*фильтрации по значениям суммы поля industry_id (таблицы employers_industries) равно 4*/ 
            order by e.name
            limit 3
            '''
df = pd.read_sql_query(query_5_5, connection)
df


c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,employer_name,industry_count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4


In [37]:
# результат запроса
# 2ГИС	

In [67]:
# поиск 5 работодателей, у которых указано больше всего видов делятельности (отраслей)
query_5_5_1 = f'''select
            e.name as employer_name,
            count(ei.industry_id) as industry_count /*подсчет уникальных значений столбца industry_id таблицы employers_industries*/
            from employers e
            left join employers_industries ei on e.id = ei.employer_id /*присоединение таблицы employers_industries ei; условие: employer_id  равно id*/
            group by e.name /*группировка по столбцу name таблицы employers*/
            order by industry_count desc
            limit 5   
            '''
df = pd.read_sql_query(query_5_5_1, connection)
df


c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,employer_name,industry_count
0,Модуль,16
1,Простые решения,11
2,Энергия,10
3,Прайд,10
4,СБК,9


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [49]:
# текст запроса
query_5_6 = f'''select
            count(distinct ei.employer_id) as employers_count /*подсчет уникальных значений столбца employer_id таблицы employers_industries*/
            from employers_industries ei
            left join industries i on ei.industry_id = i.id/*присоединение таблицы employers_industries ei; условие: employer_id  равно id*/
            where i.name like '%Разработка программного обеспечения%' /*фильтрации по полю name (таблицы industries) содержит "Разработка программного обеспечения"*/
            '''
df = pd.read_sql_query(query_5_6, connection)
df


,employers_count
0,3553


In [39]:
# результат запроса
# 3553

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [6]:
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
from lxml import etree as et

In [7]:
# код для получения списка городов-милионников
cities = pd.read_html('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')
df = cities[1]
# преобразование полученных данных в список
cities_list = df['Город'].to_list()
cities_list[13]  = 'Воронеж'
# преобразование списка городов-миллионников в кортеж
million_cities = tuple(cities_list)
# print(million_cities)

In [14]:
# текст запроса (дополнительно определили долю вакансия Яндекса в каждом городе-миллионнике)
# общее количество вакансий Яндекса (1933) определили в задании 5.1
query_5_7 = f'''(select
                a.name as name, 
                count(v.id) as cnt, /*подсчет значений столбца id таблицы vacancies*/
                round(count(v.id)*100.0 /1933, 2) as percent /*определение доли значений столбца id таблицы vacancies в общем количестве вакансий*/
                from 
                areas a 
                left join vacancies v on a.id = v.area_id /*присоединение таблицы vacancies v; условие: area_id равно id*/
                left join employers e on v.employer_id = e.id /*присоединение таблицы employers e; условие: employer_id  равно id*/
                where e.name = 'Яндекс' 
                and a.name in {million_cities} /*фильтрации по полю name (таблицы employers) равно "Яндекс" и поле name таблицы areas из списка городов-миллиоников*/
                group by a.name /*группировка по столбцу name таблицы areas*/
                order by cnt desc)
                
                union all /*присоединение*/
                
                select 
                'Total', 
                count(v.id), 
                round(count(v.id)*100.0 /1933, 2) as percent 
                from 
                vacancies v 
                left join areas a on a.id = v.area_id 
                left join employers e on v.employer_id = e.id 
                where e.name = 'Яндекс' 
                and a.name in {million_cities}
                order by cnt             
        '''
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,cnt,percent
0,Омск,21,1.09
1,Челябинск,22,1.14
2,Красноярск,23,1.19
3,Волгоград,24,1.24
4,Казань,25,1.29
5,Ростов-на-Дону,25,1.29
6,Пермь,25,1.29
7,Самара,26,1.35
8,Уфа,26,1.35
9,Краснодар,30,1.55


In [ ]:
# результат запроса
# строк запроса - 17 
# общее количесвто вакансий Яндекс во всех регионах - 485

In [8]:
# дополнительно, проанализировали долю вакансий в городах-миллиониках
query_5_7_1 = f'''(select
                a.name as name, 
                count(v.id) as quantity,
                round(count(v.id)*100.0 / (select count(*) from vacancies), 2) as percent
                from 
                areas a 
                left join vacancies v on a.id = v.area_id 
                left join employers e on v.employer_id = e.id 
                where a.name in {million_cities} 
                group by a.name
                order by quantity desc)
                
                union all
                
                select 
                'Total', 
                count(v.id),
                round(count(v.id)*100.0 / (select count(*) from vacancies), 2) 
                from 
                vacancies v 
                left join areas a on a.id = v.area_id 
                left join employers e on v.employer_id = e.id 
                where a.name in {million_cities}                
        '''
df_5_7_1 = pd.read_sql_query(query_5_7_1, connection)
df_5_7_1

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,name,quantity,percent
0,Москва,5333,10.84
1,Санкт-Петербург,2851,5.80
2,Новосибирск,2006,4.08
3,Екатеринбург,1698,3.45
4,Нижний Новгород,1670,3.39
5,Казань,1415,2.88
6,Краснодар,1301,2.64
7,Самара,1144,2.33
8,Ростов-на-Дону,1131,2.30
9,Воронеж,1063,2.16


***

# выводы по анализу работодателей

Работодатели с наибольшим количеством вакансий: 
* "Яндекс" – 1933,
* "Ростелеком" – 491, "
* "Тинькофф" – 444, 
*  "СБЕР" – 428. 

 
В 593 населенных пунктах вакансий нет. 
У 3-х компаний вакансии более чем в 100 населенных пунктах: 
* "Яндекс" – 181, 
* "Ростелеком" – 152, 
* "Спецремонт" – 116. 

Сфера деятельности не указана у 8419 компаний. У 4-х компаний сфер деятельности 10 и более: 
* "Модуль" – 16, 
* "Простые решения" – 11, 
* "Энергия", "Прайд" – 10.

Сфера деятельности "Разработка ПО" у 3 553 компаний. У "Яндекса" 25 % в городах-миллионниках. В целом, на города-миллионники приходится 48,5% вакансий. 


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [10]:
# текст запроса
query_6_1 = f'''select
            count(id) as count /*подсчет значений столбца id*/
            from vacancies
            where (lower(name) like '%data%' or lower(name) like '%данн%') /*фильтрации по полю name, содержит data  и данн*/
        '''                                  
df = pd.read_sql_query(query_6_1, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,1771


In [45]:
# результат запроса
# кол-во вакансий, имеющих отношение к данным - 1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [27]:
# текст запроса
query_6_2 = f'''select
            count(id) as count /*подсчет значений столбца id*/
            from vacancies
            where (lower(name) like '%data scientist%'
            or lower(name) like '%data science%'
            or lower(name) like '%исследователь данных%'
            or name like '%ML%' and name not ilike '%HTML%'
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
            and
            (lower(name) like '%junior%' 
            or lower(experience) like '%нет опыта%' 
            or lower(employment) like '%стажировка%') /*фильтрации поля name в соответствии с данными задания*/
            '''                                  
df = pd.read_sql_query(query_6_2, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,51


In [47]:
# результат запроса
# количество вакансия для начинающих дата-сайентистов - 51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [28]:
# текст запроса
query_6_3 = f'''select
            count(id) as count /*подсчет значений столбца id*/
            from vacancies
            where (lower(name) like '%data scientist%'
            or lower(name) like '%data science%'
            or lower(name) like '%исследователь данных%'
            or name like '%ML%' and name not ilike '%HTML%'
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
            and
            (key_skills like '%SQL%' 
            or key_skills like '%postgres%') /*фильтрации поля name в соответствии с данными задания*/
            '''                                  
df = pd.read_sql_query(query_6_3, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,201


In [49]:
# результат запроса
# у 201 вакансии для дата-сайентистов в качестве основных навыков указаны SQL или postgres 

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [29]:
# текст запроса
query_6_4 = f'''select
            count(id) as count /*подсчет значений столбца id*/
            from vacancies
            where (lower(name) like '%data scientist%'
            or lower(name) like '%data science%'
            or lower(name) like '%исследователь данных%'
            or name like '%ML%' and name not ilike '%HTML%'
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
            and
            (key_skills ilike '%Python%') /*фильтрации поля name в соответствии с данными задания*/
            '''                                  
df = pd.read_sql_query(query_6_4, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,351


In [51]:
# результат запроса
# у 351 вакансии для дата-сайентистов в качестве основного навыкав указан Python

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [15]:
# текст запроса
query_6_5 = f'''select
            round(avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1),2) as key_skills_avg_cnt  /*подсчет количества значений в столбцe key_skills*/ 
            from vacancies
            where (lower(name) like '%data scientist%'
            or lower(name) like '%data science%'
            or lower(name) like '%исследователь данных%'
            or name like '%ML%' and name not ilike '%HTML%'
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
            and key_skills is not null /*фильтрации поля name в соответствии с данными задания*/
            '''                                  
df = pd.read_sql_query(query_6_5, connection)
df


c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,key_skills_avg_cnt
0,6.41


In [53]:
# результат запроса
# среднее количесвто навыков, указываемых для датасайентиста - 6,41

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [37]:
# текст запроса
query_6_6 = f'''select
            experience as experience,
            round(avg(coalesce((salary_from + salary_to)/2, salary_to, salary_from))) as avg_salary    /*расчет средней заработной платы*/  
            from vacancies
            where (lower(name) like '%data scientist%'
            or lower(name) like '%data science%'
            or lower(name) like '%исследователь данных%'
            or name like '%ML%' and name not ilike '%HTML%'
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
            and (salary_from is not null or salary_to is not null) /*фильтрации поля name в соответствии с данными задания*/
            group by experience /*группировка по столбцу experience*/
            '''                                  
df = pd.read_sql_query(query_6_6, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


In [55]:
# результат запроса
# средняя заработная плата датасайенттиста с опытом работы от 3 до 6 лет - 243 115 руб.

In [39]:
# определение количества вакансий, подходящих для дата-сайентиста
query_6_7_0 = f'''select
            count(id) as count /*подсчет значений столбца id*/
            from vacancies
            where (lower(name) like '%data scientist%'
            or lower(name) like '%data science%'
            or lower(name) like '%исследователь данных%'
            or name like '%ML%' and name not ilike '%HTML%'
            or lower(name) like '%machine learning%'
            or lower(name) like '%машинн%обучен%')
            '''                                  
df = pd.read_sql_query(query_6_7_0, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,480


In [38]:
# дополнительно, проанализировали востребованность дата-сайентистов в различных отраслях
query_6_7 = f'''select
            i.name as industry,
            count(v.id) as quantity, /*подсчет значений столбца id таблицы vacancies*/
            round(count(v.id)*100.0/ 480, 2) as percent
            from vacancies v
            left join employers_industries ei on v.employer_id = ei.employer_id /*присоединение таблицы employers_industries ei; условие: employer_id  равно id*/
            left join industries i on ei.industry_id = i.id /* присоединение таблицы industries e; условие: industry_id  равно id*/
            where (lower(v.name) like '%data scientist%'
            or lower(v.name) like '%data science%'
            or lower(v.name) like '%исследователь данных%'
            or v.name like '%ML%' and v.name not ilike '%HTML%'
            or lower(v.name) like '%machine learning%'
            or lower(v.name) like '%машинн%обучен%') /*фильтрации поля name таблицы vacancies по ключевым словам, относящихся к дата-сайентистов*/
            group by i.name /*группировка по столбцу наименованию отрасли*/
            order by quantity desc
            limit 15
            '''
df = pd.read_sql_query(query_6_7, connection)
df

c:\Users\DOM\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,industry,quantity,percent
0,Разработка программного обеспечения,148,30.83
1,"Системная интеграция, автоматизации технологи...",139,28.96
2,Банк,92,19.17
3,"Интернет-компания (поисковики, платежные систе...",79,16.46
4,None,62,12.92
5,Мобильная связь,23,4.79
6,"Маркетинговые, рекламные, BTL, дизайнерские, E...",22,4.58
7,"Управляющая, инвестиционная компания (управлен...",21,4.38
8,Консалтинговые услуги,16,3.33
9,Фиксированная связь,13,2.71


***

# выводы по предметному анализу
Вакансий, имеющих отношение к данным – 1 771, для начинающих дата-сайентистов – 51 вакансия. Навыки SQL или Postgres требуется для 201 вакансии дата-сайентиста, Python – для 351, в среднем, для дата-сайентиста указывается 6,4 навыка. 

Разница в заработной дата-сайнетиста без опыта и с опытом от 3 до 6 лет составляет примерно 300%.

Дополнительно, проанализировали востребованность дата-сайентистов в различных отраслях, наибольшая востребованность в районе 30 % в сфере разработки ПО и системной интеграции, далее в банках и интернет компаниях 19% и 16% соответственно.

# Общий вывод по проекту

# подведем итог исследования, обобщите выводы
Всего проанализировали 49 197 вакансий от 23501 работодателей в 1632 населенных пунктах по 29 отраслям.

В ходе проведения анализа выявили, что около половины вакансий связаны напрямую с ИТ, и на города-миллионники также приходится около половины вакансий. В основном, требуются сотрудники с опытом работы (80% вакансий) на полный рабочий день с полной занятостью.

Дата-сайентисты наиболее востребованы в сфере разработки ПО, системной интеграции, банках и интернет-компаниях.

В ML-модели, предлагающей вакансии IT-специалистам, необходимо учитывать требуемые ключевые навыки, опыт, тип занятости, а также регион вакансии. Сферу деятельности компании в модели учитывать не целесообразно, т.к. в ходе дополнительных исследований по определению уровня заработной платы в различных отраслях, выявили, что заработная плата не зависит от сферы деятельности компании, а напрямую связана с навыками, необходимыми для выполнения работы. 